# Week 5 — RST documentation mentor (chatbot)

RAG pipeline: download repo `.rst` docs → chunk & embed → Chroma → answer questions with context.

**Models**
- Chunking: Gemini 2.5 Flash  
- Chat: Gemini 3 Flash  
- Eval: Gemini 2.5 Flash 
- Embeddings: EmbeddingGemma-300M

## Three scripts

| Script | Role |
|--------|------|
| **git_rst_extractor.py** | Download repo zip (tag/branch) → unzip → keep only `.rst` files. |
| **ingest.py** | Load `.rst` → chunk with LLM (headline + summary + text) → embed (SentenceTransformer) → Chroma. Collection name = last path segment (e.g. `owner_repo_tag_v1.0`). |
| **answer.py** | Retrieve top‑k chunks from Chroma → build system prompt with context → chat LLM → return answer + sources. |

**Flow:** extractor → path → `ingest(path)` → run `app.py` with same collection name for Q&A.

---

## git_rst_extractor.py

- **CLI:** `repo_url`, `ref`; optional `--ref-type {tag|branch}`, `-o/--output-dir`. Output under `output_dir/<owner>_<repo>_<ref_type>_<ref>/`.
- **From code:** `download_github_repo_rst(repo_url, ref, ref_type=..., output_dir=...)` → returns `Path` to folder with only `.rst` files.

---

## ingest.py

- **Pipeline:** `fetch_documents(path)` (subdirs → `.rst` → `{type, source, text}`) → `create_chunks(docs, repo)` (LLM returns headline/summary/original per chunk; optional `Pool`) → `create_embeddings(chunks, collection_name)` (SentenceTransformer + Chroma, batched).
- **Entry:** `ingest(repo)` or CLI: `python ingest.py <path>`.

---

## answer.py

- **Retrieval:** Chroma with same `DB_NAME` and embedding model; `collection_name=repo`; retriever `k=10`.
- **Generation:** System prompt = “technical mentor for {repo}, use context; if empty say so.” Context = concatenated retrieved chunks. Returns `(answer_text, list_of_docs)`.

---

## Usage

**CLI**
```bash
python git_rst_extractor.py https://github.com/owner/repo v1.0 --ref-type tag -o ./download
python ingest.py ./download/owner_repo_tag_v1.0
# Then run app.py; pass repo=owner_repo_tag_v1.0 for answers.
```

**From Python**
```python
from pathlib import Path
from git_rst_extractor import download_github_repo_rst
from ingest import ingest
from answer import answer_question

doc_path = download_github_repo_rst("https://github.com/owner/repo", "main", ref_type="branch", output_dir=Path("./download"))
ingest(str(doc_path))
answer, docs = answer_question("How do I configure X?", repo=doc_path.name, history=[])
```
Use the same `DB_NAME` and embedding model in ingest and answer.
